
# NoteBook Summary:
- Established connection to MySQL database.
- Created a new database.
- Created a table named "Jobs_and_Salaries_in_Data_field_2024" within "jobs_in_data" to store CSV data.
- Inserted data from CSV file into "Jobs_and_Salaries_in_Data_field_2024" table.
- Confirmed successful data insertion.



In [ ]:
%pip install mysql-connector-python
%pip install python-dotenv
%pip install pandas

In [5]:
import os
import mysql.connector
from dotenv import load_dotenv

load_dotenv()
password = os.getenv("MYSQL_ROOT_PASSWORD")
# Connect to MySQL database
db_config = {
    "host": "localhost",
    "user": "root",
    "password": password,
    "database": "world" # default database
}

try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    
except mysql.connector.Error as e:
    print(f"Error connecting to MySQL database: {e}")

In [5]:
# Read CSV file
import pandas as pd

df = pd.read_csv("data/jobs_in_data_2024.csv.zip", compression="zip")
df.head(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,work_setting,company_location,company_size,job_category
0,2024,Entry-level,Freelance,Applied Data Scientist,30000,USD,30000,United Kingdom,Remote,United Kingdom,M,Data Science and Research
1,2024,Executive,Full-time,Business Intelligence,230000,USD,230000,United States,In-person,United States,M,BI and Visualization
2,2024,Executive,Full-time,Business Intelligence,176900,USD,176900,United States,In-person,United States,M,BI and Visualization


In [6]:
# Check column names
df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'work_setting', 'company_location', 'company_size', 'job_category'],
      dtype='object')

In [7]:
# Check shape
df.shape

(14199, 12)

In [8]:
# Check missing values
df.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
work_setting          0
company_location      0
company_size          0
job_category          0
dtype: int64

In [9]:
# Check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14199 entries, 0 to 14198
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           14199 non-null  int64 
 1   experience_level    14199 non-null  object
 2   employment_type     14199 non-null  object
 3   job_title           14199 non-null  object
 4   salary              14199 non-null  int64 
 5   salary_currency     14199 non-null  object
 6   salary_in_usd       14199 non-null  int64 
 7   employee_residence  14199 non-null  object
 8   work_setting        14199 non-null  object
 9   company_location    14199 non-null  object
 10  company_size        14199 non-null  object
 11  job_category        14199 non-null  object
dtypes: int64(3), object(9)
memory usage: 1.3+ MB


In [10]:
# Create a new database
new_database_name = "jobs_in_data"
create_database_query = f"CREATE DATABASE IF NOT EXISTS {new_database_name}"

try:
    cursor.execute(create_database_query)
    print(f"Database '{new_database_name}' created successfully.")

except mysql.connector.Error as e:
    print(f"Error creating database: {e}")


Database 'jobs_in_data' created successfully.


In [11]:
# switch to new database
db_config["database"] = new_database_name
conn.close()

# connect to new database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

In [13]:
# Create a table to store the data
create_table_query = """
CREATE TABLE IF NOT EXISTS Jobs_and_Salaries_in_Data_field_2024 (
    work_year INT,
    experience_level VARCHAR(255),
    employment_type VARCHAR(255),
    job_title VARCHAR(255),
    salary INT,
    salary_currency VARCHAR(255),
    salary_in_usd INT,
    employee_residence VARCHAR(255),
    work_setting VARCHAR(255),
    company_location VARCHAR(255),
    company_size VARCHAR(255),
    job_category VARCHAR(255)
    
)
"""

try:
    cursor.execute(create_table_query)
    print("Table 'Jobs_and_Salaries_in_Data_field_2024' created successfully.")

except mysql.connector.Error as e:
    print(f"Error creating table: {e}")

Table 'Jobs_and_Salaries_in_Data_field_2024' created successfully.


In [14]:
# Insert data into the table
insert_query = "INSERT INTO Jobs_and_Salaries_in_Data_field_2024 values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

for index, row in df.iterrows():
    cursor.execute(insert_query, tuple(row)) # The `execute()` method of the MySQL cursor expects data to be provided in the form of a tuple.

conn.commit()
print("Data inserted successfully.")

# Close the connection
conn.close()

Data inserted successfully.


### We want to execute SQL queries directly to validate the responses from the AI assistant

In [6]:
# Connect to the new database
db_config["database"] = "jobs_in_data"

try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
except mysql.connector.Error as e:
    print(f"Error connecting to MySQL database: {e}")


#### 1. Find the average salary for each job category

In [ ]:
%pip install tabulate # library to format the output as a table

In [7]:
from tabulate import tabulate

# Select data from the table
select_query = """

SELECT job_category, AVG(salary) AS average_salary
FROM Jobs_and_Salaries_in_Data_field_2024
GROUP BY job_category;

"""

try:
    cursor.execute(select_query)
    # Fetch all rows
    rows = cursor.fetchall()
    
    # print result in table format
    print(tabulate(rows, headers=["job_category", "average_salary"], tablefmt="grid"))

except mysql.connector.Error as e:
    print(f"Error executing query: {e}")

+--------------------------------+------------------+
| job_category                   |   average_salary |
+================================+==================+
| Data Science and Research      |         162092   |
+--------------------------------+------------------+
| BI and Visualization           |         125778   |
+--------------------------------+------------------+
| Data Architecture and Modeling |         155835   |
+--------------------------------+------------------+
| Data Analysis                  |         106053   |
+--------------------------------+------------------+
| Data Engineering               |         146131   |
+--------------------------------+------------------+
| Leadership and Management      |         141402   |
+--------------------------------+------------------+
| Data Quality and Operations    |          95237.8 |
+--------------------------------+------------------+
| Machine Learning and AI        |         182150   |
+---------------------------

#### 2. Identify the top 5 highest paying job titles

In [9]:
# Select data from the table
select_query = """
SELECT job_title, MAX(salary) AS max_salary
FROM Jobs_and_Salaries_in_Data_field_2024
GROUP BY job_title
ORDER BY max_salary DESC
LIMIT 5;

"""
try:
    cursor.execute(select_query)
    # Fetch all rows
    rows = cursor.fetchall()
    
    # print result in table format
    print(tabulate(rows, headers=["job_title", "max_salary"], tablefmt="grid"))
except mysql.connector.Error as e:
    print(f"Error executing query: {e}")


+------------------------------------+--------------+
| job_title                          |   max_salary |
+====================================+==============+
| Research Engineer                  |       450000 |
+------------------------------------+--------------+
| Research Scientist                 |       450000 |
+------------------------------------+--------------+
| Head of Machine Learning           |       448000 |
+------------------------------------+--------------+
| ML Engineer                        |       440000 |
+------------------------------------+--------------+
| Applied Machine Learning Scientist |       423000 |
+------------------------------------+--------------+


#### 3. Count the number of jobs in each employment type

In [10]:
select_query = """

SELECT employment_type, COUNT(*) AS job_count
FROM Jobs_and_Salaries_in_Data_field_2024
GROUP BY employment_type;


"""

try:
    cursor.execute(select_query)
    # Fetch all rows
    rows = cursor.fetchall()
    
    # print result in table format
    print(tabulate(rows, headers=["employment_type", "job_count"], tablefmt="grid"))

except mysql.connector.Error as e:
    print(f"Error executing query: {e}")

+-------------------+-------------+
| employment_type   |   job_count |
+===================+=============+
| Freelance         |          12 |
+-------------------+-------------+
| Full-time         |       14139 |
+-------------------+-------------+
| Contract          |          26 |
+-------------------+-------------+
| Part-time         |          22 |
+-------------------+-------------+


#### 4. Calculate the average salary for different experience levels

In [11]:
select_query = """

SELECT experience_level, AVG(salary) AS average_salary
FROM Jobs_and_Salaries_in_Data_field_2024
GROUP BY experience_level;


"""

try:
    cursor.execute(select_query)
    # Fetch all rows
    rows = cursor.fetchall()
    
    # print result in table format
    print(tabulate(rows, headers=["experience_level", "average_salary"], tablefmt="grid"))

except mysql.connector.Error as e:
    print(f"Error executing query: {e}")

+--------------------+------------------+
| experience_level   |   average_salary |
+====================+==================+
| Entry-level        |          91714.2 |
+--------------------+------------------+
| Executive          |         191626   |
+--------------------+------------------+
| Senior             |         162948   |
+--------------------+------------------+
| Mid-level          |         122983   |
+--------------------+------------------+


#### 5. Find the five highest paying company locations

In [13]:
select_query = """
SELECT company_location, MAX(salary) AS max_salary
FROM Jobs_and_Salaries_in_Data_field_2024
GROUP BY company_location
ORDER BY max_salary DESC
LIMIT 5;

"""

try:
    cursor.execute(select_query)
    # Fetch all rows
    rows = cursor.fetchall()
    
    # print result in table format
    print(tabulate(rows, headers=["company_location", "max_salary"], tablefmt="grid"))

except mysql.connector.Error as e:
    print(f"Error executing query: {e}")


+--------------------+--------------+
| company_location   |   max_salary |
+====================+==============+
| United States      |       450000 |
+--------------------+--------------+
| Canada             |       409500 |
+--------------------+--------------+
| Türkiye            |       360000 |
+--------------------+--------------+
| United Kingdom     |       350000 |
+--------------------+--------------+
| Australia          |       300000 |
+--------------------+--------------+


#### 6. Top 10 Highest-Paying Full-Time Senior Positions

In [14]:
select_query = """
SELECT job_title, AVG(salary) AS average_salary
FROM Jobs_and_Salaries_in_Data_field_2024
WHERE experience_level = 'Senior' 
  AND employment_type = 'Full-time'
GROUP BY job_title
HAVING COUNT(*) > 5
ORDER BY average_salary DESC
LIMIT 10;

"""

try:  
    cursor.execute(select_query)
    # Fetch all rows
    rows = cursor.fetchall()
    
    # print result in table format
    print(tabulate(rows, headers=["job_title", "average_salary"], tablefmt="grid"))

except mysql.connector.Error as e:
    print(f"Error executing query: {e}")

+------------------------------------+------------------+
| job_title                          |   average_salary |
+====================================+==================+
| Deep Learning Engineer             |           295890 |
+------------------------------------+------------------+
| AI Architect                       |           230348 |
+------------------------------------+------------------+
| Data Analytics Lead                |           221548 |
+------------------------------------+------------------+
| Machine Learning Software Engineer |           218655 |
+------------------------------------+------------------+
| ML Engineer                        |           215841 |
+------------------------------------+------------------+
| Computer Vision Engineer           |           209017 |
+------------------------------------+------------------+
| Data Science Manager               |           202197 |
+------------------------------------+------------------+
| Data Infrast